Machine Learning - Homework 1 (due Sep. 13)

Problem 1: Legal reasoning (from Murphy 2.2).

Suppose a crime has been committed. Blood is found at the scene for which there is no innocent explanation. It is of a type which is present in 1% of the population.  The defendant is known to have this rare blood type.

a. The prosecutor claims: “There is a 1% chance that the defendant would have the crime blood type if he
were innocent. Thus there is a 99% chance that he guilty”. This is known as the prosecutor’s fallacy.
What is wrong with this argument?

______________________________
$\textbf{Answer}$

If 1% of the population share the criminal's blood type, then even in a modest community of only 1000 people, chances are there would be 10 individuals with the suspect blood type. By the judge's logic, each of these 10 people has a 99% chance of guilt, which is of course impossible, because the sum of everybody's probability of guilt must add up precisely to 1. 
______________________________

b. The defender claims: “The crime occurred in a city of 800,000 people. The blood type would be
found in approximately 8000 people. The evidence has provided a probability of just 1 in 8000 that
the defendant is guilty, and thus has no relevance.” This is known as the defender’s fallacy. What is
wrong with this argument (HINT: What happens to the prior in this case if there is *other* evidence presented)?

___
$\textbf{Answer}$

By consulting Bayes' theorem, we can demonstrate that the likelihood of the defendant's guilt is not 1 in 8000, even absent some of the values necessary to precisely calculate the probability of guilt. In the following, $G$ is the event of the defendant's guilt, and $B$ is the event of the defendant's having the same blood type as the perpetrator.

$$
P(G=1|B=1) = \frac{P(B=1|G=1)P(G=1)}{P(B=1)}
$$

The probability that the guilty party has the same blood type as, well, the guilty party, is 1 ( $P(B|G)=1$ ). 

The probabiliy of guilt is the reciprical of the number of possible guilty people, that being the number of people with the criminal's blood type ( $P(G=1)=\frac{1}{8000}=0.000125$ ). 

The probability of having the criminal blood type is 1% ( $P(B=1)=0.01$ ).

Therefore, having gathered all relevant probabilities, we can assess $P(G=1|B=1)$.

$$
P(G=1|B=1) = \frac{(1)(0.000125)}{0.01} = 0.0125 
$$

This is much higher than the defender's claim of a 0.000125 probability of guilt. Though one could certainly argue that the equation fails to address a number of important variables that factor into the probability of guilt, it is still a much more rigorous and accurate method than that of the defender's.
___

c. Suppose that forensic analysis tells us that that the blood test has 98% sensitivity (true positive rate) and a 1% false positive rate.  Given the information presented in the above two questions, determine the posterior probability the the defendent is guilty, given that the defendent's blood type matches that found at the crime scene *and* that the defendent was one of only 5 people with access to the crime scene *and* that there is no other evidence.  

___
$\textbf{Answer}$

The probability can be calculated via Bayes' theorem:

$$
P(G=1|B=1) = \frac{P(B=1|G=1)P(G=1)}{P(B=1)}
$$

$P(B=1|G=1)$ is the probability of a positive blood match if the defendant does in fact have the suspect blood type. This is precisely the definition of the test's sensitivity, so $P(B=1|G=1)=0.98$.

$P(G=1)$ is the prior probability of guilt. Because the defandant is one of five possible offendors, $P(G=1)=\frac{1}{5}$, or $0.20$.

$P(B=1)$ is the overall probability of a positive result of the blood test. This is equal to the sum of the probabilities of a true positive and a false positive, or $P(B=1) = P(B=1|G=1)P(G=1) + P(B=1|G=0)P(G=0) = 0.98 * 0.20 + 0.01 * 0.80 = 0.204$, where $M$ is the event in which the blood is in fact a match for the target type.

Having determined all relevant values, we can calculate the final value of Bayes' theorem:

$$
P(G=1|B=1) = \frac{0.98 * 0.20}{0.204} = 0.9608
$$
___

Problem 2: Naive Bayes.

One use of the naive Bayes classifier, which is still in practical use today, is as a spam filter.  Consider the corpus of text messages packaged with this homework, which are each labelled as either 'spam' or 'ham'.  In this case, naive Bayes utilizes a Bernoulli model that quantifies the probability of a given word given that the message is either spam or ham.  For example, investigating the text messages here, we find that the word *draw* shows up in spam 27 times, yet in ham only 5.  Thus, we have that
$$ P(X=\mathrm{draw}|Y=\mathrm{ham}) = \frac{5}{5+27}. $$

While this is not particularly strong evidence on its own, we can create a powerful classifier by using the naive assumption in conjunction with all the words in a given message:
$$ P(Y=\mathrm{ham}|X=x) \propto P(Y=\mathrm{ham}) \prod_{i=1}^n P(X_i=x_i|Y=\mathrm{ham}), $$
$$ P(Y=\mathrm{spam}|X=x) \propto P(Y=\mathrm{spam}) \prod_{i=1}^n P(X_i=x_i|Y=\mathrm{spam}), $$
where $x_i$ are the words in a given message. 

Your task is to write such a classifier.  I have taken the somewhat tedious step of parsing the data for you, yielding the variables *word_dictionary*, which contains the ham and spam counts for each word, as well as *training_labels*, which provides the spam/ham labels for each text message.  I have also parsed a set of test data: *test_messages* is a list, each entry containing another list of the words in the text message, as well as *test_labels* which contains the spam/ham label for each message.

In [86]:
import numpy as np

# Maps from 'ham' or 'spam' strings to zero or one
def mapper(s):
    if s=='spam':
        return 0
    else:
        return 1

# Read in the text file
f = open('SMSSpamCollection','r')
lines = f.readlines()

# Break out the test data
test_lines = lines[:len(lines)//5]
lines = lines[len(lines)//5:]

# Instantiate the frequency dictionary and an array to
# record whether the line is ham or spam
word_dictionary = {}
training_labels = np.zeros(len(lines),dtype=int)

# Loop over all the training messages
for i,l in enumerate(lines):
    # Split into words
    l = l.lower().split()
    # Record the special first word which always ham or spam
    if l[0]=='ham':
        training_labels[i] = 1
    # For each word in the message, record whether the message was ham or spam
    for w in l[1:]:
        # If we've never seen the word before, add a new dictionary entry
        if w not in word_dictionary:
            word_dictionary[w] = [1,1]
        word_dictionary[w][mapper(l[0])] += 1
        
# Loop over the test messages
test_labels = np.zeros(len(test_lines),dtype=int)
test_messages = []
for i,l in enumerate(test_lines):
    l = l.lower().split()
    if l[0]=='ham':
        test_labels[i] = 1
    test_messages.append(l)


Below, I have provided code skeletons.  Your job is to make the code skeletons into an operational naive Bayes spam detector.  (you may discard these skeletons if you would prefer to code this from scratch).  Note that lines where you will need to change the code are marked with a '#!'.

Your first task is train the model:  

In [87]:
#What is the prior P(Y=ham) ?
ham_prior = np.sum(training_labels) / len(training_labels)
spam_prior = 1 - ham_prior

# What are the class probabilities P(X=word|Y=ham) for each word?
ham_likelihood = {}
for key,val in word_dictionary.items():
    ham_likelihood[key] = val[1] / (val[0] + val[1])

Your next task is to make predictions on a set of test examples which were held back from the training procedure (see *test_messages* variable).  For each of these messages, compute the ham and spam probabilities.

In [88]:
# Where to hold the ham and spam posteriors
posteriors = np.zeros((len(test_lines),2))


# Loop over all the messages in the test set
for i,m in enumerate(test_messages):
    posterior_ham = ham_prior
    posterior_spam = spam_prior
    #! Don't forget to include the prior!
    
    # Loop over all the words in each message
    for w in m:
        if(w == "ham" or w == "spam"):
            continue
        # #! What is the purpose of this try/except handler?
        # This is to prevent the script from throwing an error in the event that  
        # we try to access a word in the 'ham_likelihood' dictionary that was not 
        # encountered in our training data.
        try:
            posterior_ham *= ham_likelihood[w]
            posterior_spam *= (1.0-ham_likelihood[w]) #!
        except KeyError:
            pass
    
    # Notice the normalization factor (denominator) 
    # to turn these into proper probabilities!
    posteriors[i,0] = posterior_spam/(posterior_spam + posterior_ham)
    posteriors[i,1] = posterior_ham/(posterior_spam + posterior_ham)

Finally, make a ham/spam prediction based on your posterior probabilities.  Compare these to the labels contained in test_labels.  Report the accuracy of your classifier as percentage correct.

In [89]:
correct = 0
total = 0

for i,m in enumerate(test_messages):
    actual_class = mapper(m[0])
    predicted_class = np.argmax(posteriors[i])
    
    if(actual_class == predicted_class):
        correct += 1
    
    total += 1
    
print("Messages were correctly classified %.2f%% of the time!" % (correct/total*100))

    

Messages were correctly classified 93.81% of the time!
